In [ ]:
from difflib import SequenceMatcher
import pandas as pd
from Levenshtein import distance
import Levenshtein

In [ ]:
!pip install Levenshtein

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.4/177.4 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 38.6 MB/s eta 0:00:00


In [ ]:
reci = pd.read_csv('/content/만개의레시피수정.csv')
reci.head()

,음식명,CKG_NM,CKG_KND_ACTO_NM,CKG_MTRL_ACTO_NM,CKG_STA_ACTO_NM,CKG_MTH_ACTO_NM,CKG_MTRL_CN,CKG_INBUN_NM,CKG_DODF_NM,Column4,CKG_IPDC
0,오이 소박이 무침 황금 레시피/오이 김치 담그는 법/오이 부추 무침,오이소박이,김치/젓갈/장류,채소류,일상,무침,[재료] 오이 10개| 굵은소금(손질/절임용) 12T| 부추 1/2단| 양파 1개|...,6인분이상,중급,1. 오이는 소금(6큰술)로 겉을 충분히 문질러 씻은 후 행궈주세요\n2. 양끝을 ...,아삭아삭 맛있는 오이 소박이 김치를 만들었어요 넉넉히 만들어두니 매끼 밥 한그릇 ...
1,백종원레시피로 만든 콩나물무침으로 밥 한 끼 뚝딱 ~,콩나물무침,밑반찬,채소류,초스피드,무침,[재료] 콩나물 1봉지| 대파(또는 쪽파) 1/6대| 당근 1/5개 [양념] 고춧가...,2인분,아무나,1. 콩나물은 흐르는 물에 여러 번 조심스레 씻어준 뒤 체에 밭쳐 물기를 빼둡니다....,오늘은 오래간만에 고춧가루 넣은 콩나물무침을 만들어봤어요. 아삭아삭 식감에 칼칼한 ...
2,두부조림 양념장 만드는 법,두부조림,밑반찬,콩/견과류,일상,조림,[재료 ] 두부 1모| 물 1/4컵| 소금 약간 [양념] 고춧가루 1스푼| 간장 3...,2인분,아무나,1. 먹기 좋게 한 입 크기로 도톰하게 썰어주고 키친타월을 이용해 물기를 제거하고 ...,두부조림
3,절대 실패없는 제육볶음 황금레시피 감칠맛과 매운맛이 좋아요~!!,제육볶음,메인반찬,돼지고기,초스피드,볶음,[재료] 돼지고기 앞다리살 600g| 양파 1/2개| 청양고추 2~3개| 대파 1/...,3인분,초급,1. 재료를 준비해주세요~\n2. 양념재료를 모두 넣어주세요~\n3. 양념재료가 잘...,밥과 함께 먹으면 제일 맛있는 제육볶음 황금레시피를 준비해보았어요~!! 상추와 함께...
4,돼지고기 김치찌개 맛내는 비법,돼지고기김치찌개,찌개,돼지고기,일상,끓이기,[재료] 물 2컵(250ml 기준)| 돼지고기 찌개용(or 목살) 250g| 신김치...,3인분,초급,1. 돼지고기는 분량의 양념을 넣어 조물조물 해주신후 30분 이상 재웁니다.\n2....,진한 국물이 맛있는 돼지고기 김치찌개입니다.


In [ ]:
# 빈 딕셔너리 초기화
chat_dic = {}
row = 0

# 'CKG_NM' 항목을 '|' 문자를 기준으로 분할하여 리스트로 변환 후 딕셔너리에 저장
for rule in reci['CKG_NM']:
    chat_dic[row] = rule.split('|')
    row += 1

In [ ]:
# Levenshtein 거리를 사용하여 가장 유사한 CKG_NM 값을 찾고 응답을 반환하는 함수
def chat(request):
    max_similarity = 0
    best_match = None
    best_index = -1

    # 입력 텍스트와 CKG_NM 값을 비교하여 가장 유사한 값을 찾음
    for k, v in chat_dic.items():
        for word in v:
            similarity = 1 - (distance(request, word) / max(len(request), len(word)))
            if similarity > max_similarity:
                max_similarity = similarity
                best_match = word
                best_index = k

    # 가장 유사한 CKG_NM 값에 대응하는 응답 텍스트 생성
    if best_match and max_similarity > 0.5:  # 유사도가 50% 이상인 경우만 응답 생성
        return reci['CKG_NM'][best_index] + '\n' + reci['CKG_MTRL_CN'][best_index] + '\n' + reci['Column4'][best_index]

    return '음식명으로 검색해주세요.. 데헷'

In [ ]:
# 예시 입력
while True:
    req = input("채팅을 입력하세요! (종료하려면 '끝'을 입력하세요): ")
    if req == '끝':
        break
    else:
        print("쿠킹파트너 : ", chat(req))

채팅을 입력하세요! (종료하려면 '끝'을 입력하세요): 오이소박이 알려줘
쿠킹파트너 :  오이소박이
[재료] 오이 10개| 굵은소금(손질/절임용) 12T| 부추 1/2단| 양파 1개| 물 12컵 [양념재료] 굵은고춧가루 1(종이)컵| 다진마늘 1/2(종이)컵| 굵은소금 2T| 설탕 2T| 멸치액젓 4T| 다진생강 2t
1. 오이는 소금(6큰술)로 겉을 충분히 문질러 씻은 후 행궈주세요
2. 양끝을 잘라낸 후 3~4토막 내주세요
3. 오이 토막은 밑부분 1.5cm 정도만 남기고 열십(+)자로 칼집을 넣어주세요
4. 냄비에 오이절임용 물과 소금 6T를 넣고 끓여 주세요
5. 볼에 손질한 오이를 담고, 끓는 소금물을 부어 40분간 그대로 두고, 찬물에 헹군후 물기를 빼줍니다
6. 오이가 절여지는 동안, 소박이소를 만들어볼께요. 부추는 깨끗하게 씻어 물기를 제거한 후 1.5cm 길이로 썰고, 양파는 얇게 채썰어 줍니다
7. 볼에 부추를 담고 다진 생강2t,
8. 멸치액젓 4T,
9. 다진마늘 4T,
10. 소금 2T,
11. 설탕 2T,
12. 굵은 고춧가루 한컵을 모두 넣고 야채와 골고루 버무려주세요
13. 오이소박이소 완성! 이제 절인 오이에 소박이소를 적당히 채워 넣어 주시면 끝이에요 :)
14. 오이를 통에담아, 실온에서 3시간-1일 정도 숙성한 후, 맛이 들면 냉장고에 넣어 주세요 이렇게 보니, 오이 10개 정말 많네요~~ 부모님댁에 드리고 우리도 먹고:) 흐흐
15. 요아이, 정말 너무 맛있어요 :) 끓는 소금물을 부어서, 아삭한 맛이 오래가는, 상큼한 오이 김치! 우리 이웃님들도 한번 도전해 보세요 ^3^ 우리 함께 맛있고 행복해져요! 저는 다시 올게요 한주의 시작 즐겁게 보내요 씨유쑨! ♥
채팅을 입력하세요! (종료하려면 '끝'을 입력하세요): 오이 소박이 레시피 알아?
쿠킹파트너 :  음식명으로 검색해주세요.. 데헷
채팅을 입력하세요! (종료하려면 '끝'을 입력하세요): 오이 소박이
쿠킹파트너 :  오이소박이
[재료] 오이 10개| 굵은소금(손질/절임